# I. Importation des bibliothèques


In [ ]:
%%capture
%pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib unidecode

In [ ]:
%%capture
import warnings

warnings.filterwarnings("ignore")

import os
from pathlib import Path

import pandas as pd
from openhexa.sdk import workspace
from unidecode import unidecode

pd.set_option("display.max_columns", None)

try:
    import openpyxl as pyxl
    from efc.interfaces.iopenpyxl import OpenpyxlInterface
    from fuzzywuzzy import fuzz, process
except ImportError or ModuleNotFoundError:
    %pip install --quiet fuzzywuzzy python-Levenshtein excel-formulas-calculator
    import openpyxl as pyxl
    from efc.interfaces.iopenpyxl import OpenpyxlInterface

## 1. Rafraîchissement des bibliothèques


In [3]:
%%capture
# Ajout du chemin d'accès pour l'importation des bibliothèques
os.chdir(Path(workspace.files_path, "Fichier Suivi de Stock/code/pipelines"))

from importlib import reload

import compute_indicators
import export_file_to_google_drive as ggdrive
import generate_stock_tracking_file as gstf

# Importation des réquêtes sql
from compute_indicators.queries import *
from database_operations import stock_sync_manager

# Reload modules
reload(compute_indicators)
reload(stock_sync_manager)
reload(gstf)
reload(ggdrive)

# II. Définition des paramètres


## 1. Variables requises pour la conception du fichier de suivi de stock

1. **Mois de création du rapport** : Si le processus est bien défini, cette valeur peut être automatisée. Sinon, elle sera fournie en tant qu'entrée du pipeline.

2. **Programme** : Le programme concerné pour lequel le fichier de suivi de stock est généré.

3. **Fichier État Mensuel** : Fichier transmis mensuellement par la N-PSP (source : SAGE X3).

4. **Fichier Plan d'Approvisionnement** : Utilisé pour certains calculs, notamment dans la feuille « Prévision ». Ce fichier est généralement récupéré depuis la source QAT.

5. **Fichier de mappage des produits** : Fichier assurant la correspondance entre les produits de SAGE X3 et ceux de QAT.


In [4]:
(
    month_report,
    year_report,
    programme,
    fp_etat_mensuel,
    fp_plan_approv,
    fp_map_prod,
    auto_computed_dmm,
    auto_computed_cmm,
) = (
    "Août",
    2025,
    "PNLP",
    "Etat du stock et de distribution PNLP fin Août 2025.xlsx",
    "Détails de l`envoiJan. 2025 ~ Dec. 2026 - Extraction Septembre 2025.csv",
    "Mapping QAT_SAGEX3_AOUT_2025.xlsx",
    False,
    True,
)

In [ ]:
month_export, date_report = (
    month_report,
    compute_indicators.utils.format_date(month_report, year_report),
)

date_report_prec = (pd.to_datetime(date_report).replace(day=1) - pd.offsets.MonthBegin()).strftime(
    "%Y-%m-%d"
)

## 2. Test pour s'assurer que cette date n'existe pas déjà dans la base de données


In [6]:
stock_sync_manager.initialize_database_connection()

schema_name = "suivi_stock"

Connexion à la base de données établie avec succès


In [ ]:
# Ici il faudrait également s'assurer que le mois précédent est bien présent à l'intérieur de la base de données au cas où on arriverait à se tromper sur le mois de conception du fichier
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select * 
    from {schema_name}.stock_track st 
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report_prec}'
    limit 2
    """
)

assert df_.shape[0] != 0, (
    f"Le mois précédent {date_report_prec} n'est pas présent dans la base de données locale êtes-vous sûre d'avoir choisir le bon mois de conception du fichier."
)

del df_

# III.📥Importation des Données

L'utilisateur doit veiller à ce que les fichiers respectent le format attendu et soient placés dans les répertoires dédiés avant de procéder au traitement.


## 📌1. Importation du fichier `Etat de Stock Mensuel`

- **Format requis :** Assurez-vous que le fichier respecte le template standard défini.
- **Emplacement du fichier :** Le fichier doit être placé dans le répertoire dédié :  
  **`Fichier Suivi de Stock/data/<programme>/Etat de Stock Mensuel`**
- **En cas d'erreur :**
  - Vérifiez que le fichier est bien présent dans le répertoire.
  - Assurez-vous que toutes les colonnes requises sont bien renseignées.
  - Contrôlez que le fichier est bien accessible et non corrompu.


In [8]:
fp_etat_mensuel = Path(fp_etat_mensuel)

# (
#    Path(workspace.files_path)
#    / f"Fichier Suivi de Stock/data/{programme}/Etat de Stock Mensuel"
#    / Path(fp_etat_mensuel).name
# )

sheetnames = pd.ExcelFile(fp_etat_mensuel).sheet_names

## 📌2. Importation de la feuille `Etat de stock de la NPSP`


In [ ]:
sheet_stock_npsp = compute_indicators.utils.check_if_sheet_name_in_file("Etat de stock", sheetnames)

assert sheet_stock_npsp is not None, print(
    f"La feuille `Etat de stock` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_etat_stock_npsp = pd.read_excel(fp_etat_mensuel, sheet_name=sheet_stock_npsp, skiprows=4)
df_etat_stock_npsp.columns = df_etat_stock_npsp.columns.str.strip()

del sheet_stock_npsp

df_etat_stock_npsp = df_etat_stock_npsp.loc[df_etat_stock_npsp["Nouveau code"].notna()]

df_etat_stock_npsp = compute_indicators.file_utils.process_etat_stock_npsp(
    df_etat_stock_npsp, date_report, programme
)

df_etat_stock_npsp.head(2)

,date_report,code_produit,designation,contenance,dmm,traceurs,stock_theorique_bke,stock_theorique_abj,stock_theorique_central,stock_theorique_fin_mois,msd,statut_stock,programme
1,2025-08-01,3010049.0,PARACETAMOL 100 mg comp. BTE/100,BOITE/100,5384.888889,1.0,8923.0,61447.0,0.0,70370.0,13.068051,Surstock,PNLP
2,2025-08-01,3010062.0,PARACETAMOL 250 mg comp BTE/100,BOITE/100,7636,1.0,43475.0,26197.0,0.0,69672.0,9.124149,Surstock,PNLP


## 📌3. Importation de la feuille `Stock detaille`


In [ ]:
sheet_stock_detaille = compute_indicators.utils.check_if_sheet_name_in_file(
    "Stock detaille", sheetnames
)

assert sheet_stock_detaille is not None, print(
    f"La feuille `Stock detaille` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_stock_detaille = pd.read_excel(fp_etat_mensuel, sheet_name=sheet_stock_detaille)
df_stock_detaille.columns = df_stock_detaille.columns.str.strip()

max_date_year = pd.Timestamp.max.year

df_stock_detaille["Date limite de consommation"] = df_stock_detaille[
    "Date limite de consommation"
].apply(lambda x: x if x.year < max_date_year else x.replace(year=max_date_year - 1))

del sheet_stock_detaille, max_date_year

df_stock_detaille.head(2)

,Code produit,Désignation,Emplacement,Date limite de consommation,Numéro Lot,Sous lot (Programme),Qté \nPhysique,Qté \nlivrable,Unit,Site,Dépôt,Sous lot,Site.1,Bailleur,Statut
0,4150558,MILDA STANDARD/DELTAMETHRINE,27Q00A00,2099-01-01,NEANT,PNLP,3245,0,UN,NPSP,PGY,PNLP,PG05,BAILLEURX,RA
1,4150554,MILDA IG2 BALLE/50,27Q00A00,2028-01-01,I25225G21,PNLP,2677,2677,BALLE,NPSP,PGY,PNLP,PG05,FM,A


## 📌4. Importation de la feuille `Distribution X3`


In [ ]:
sheet_distribution_x3 = compute_indicators.utils.check_if_sheet_name_in_file(
    "Distribution", sheetnames
)

assert sheet_distribution_x3 is not None, print(
    f"La feuille `Distribution X3` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_distribution = pd.read_excel(fp_etat_mensuel, sheet_name=sheet_distribution_x3)
df_distribution.columns = df_distribution.columns.str.strip()

del sheet_distribution_x3

df_distribution.head(2)

,Programme,Date de livraison,No commande,No livraison,Client,Raison sociale,Article,Désignation,Unité vente,Qté commandée,Qté livrée
0,PNLP,2025-08-04,VAL01-PSG-2507-1551,BL01-PSG-2508-0005,31800014,CHR ABENGOUROU,4030177,GANTS D'EXAMEN LATEX MM PQT/100 PQT -,PQT,2,2
1,PNLP,2025-08-04,VAL01-PSG-2507-1551,BL01-PSG-2508-0005,31800014,CHR ABENGOUROU,3050471,DIHYDROARTEMISININE / PIPERAQUINE PHOSPHATE 80...,BTE,35,35


## 📌5. Importation de la feuille `Receptions`


In [ ]:
sheet_reception = compute_indicators.utils.check_if_sheet_name_in_file("Receptions", sheetnames)

assert sheet_reception is not None, print(
    f"La feuille `Receptions` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_receptions = pd.read_excel(fp_etat_mensuel, sheet_name=sheet_reception)
df_receptions.columns = df_receptions.columns.str.strip()

df_receptions["Date_entree_machine"] = pd.to_datetime(
    df_receptions["Date d'entrée en machine"], format="%d/%m/%Y", errors="coerce"
)

df_receptions["Nouveau code"] = pd.to_numeric(
    df_receptions["Nouveau code"], errors="coerce"
).astype(float)

del sheet_reception

df_receptions.head(2)

,Programme,Bailleur (Funding),Nouveau code,Code,Désignation NPSP,Date de réception effective,Date effective de dépotage et/ou expertise,Quantité réceptionnée,Date d'entrée en machine,Date_entree_machine
0,PNLP,PMI,3050498.0,NaN,ARTEMETHER/LUMEFANTRINE 80 mg/480 mg PLAQ/6 co...,2025-08-08,2025-08-11,160000,2025-08-21 00:00:00,2025-08-21
1,PNLP,FONDS MONDIAL,3050016.0,NaN,AMODIAQUINE/ARTESUNATE 50 / 135 mg ENFANT (1 -...,2025-08-13,2025-08-14,5702,2025-08-14 00:00:00,2025-08-14


## 📌6. Importation de la feuille `PPI`


In [ ]:
sheet_ppi = compute_indicators.utils.check_if_sheet_name_in_file("PPI", sheetnames)

assert sheet_ppi is not None, print(
    f"La feuille `PPI` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_ppi = pd.read_excel(fp_etat_mensuel, sheet_name=sheet_ppi, skiprows=2)
df_ppi.columns = df_ppi.columns.str.strip()

del sheet_ppi

df_ppi.head(2)

,Code Produit,Nom Produit,Unite,Numéro Lot,Date Peremption,Quantité
0,3050497,AMODIAQUINE/ARTESUNATE 50MG/135MG COMP PLQ 3,PLAQUETTE,CYY253021,2025-01-01,22
1,4030177,GANTS D'EXAMEN LATEX MM PQT/100,PQT,1304,2028-06-01,4


## 📌7. Importation de la feuille `Prélèvement`


In [ ]:
sheet_prelev = compute_indicators.utils.check_if_sheet_name_in_file("Prelèvement CQ", sheetnames)

assert sheet_prelev is not None, print(
    f"La feuille `Prelèvement CQ` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_prelevement = pd.read_excel(fp_etat_mensuel, sheet_name=sheet_prelev, skiprows=2)
df_prelevement.columns = df_prelevement.columns.str.strip()

del sheet_prelev

df_prelevement.head(2)

,Code Produit,Nom Produit,Unite,Numéro Lot,Date Peremption,Quantité


## 📌8. Importation du fichier de `mapping des produits QAT et SAGE X3`

- **Format requis :** Respectez la structure du fichier.
- **Emplacement du fichier :** Le fichier doit être placé dans le répertoire suivant :  
  **`Fichier Suivi de Stock/data/Mapping produits QAT SAGE X3`**
- **En cas d'erreur :**
  - Vérifiez que le fichier est bien présent dans le répertoire.
  - Assurez-vous que les données respectent le format attendu.
  - Vérifiez l'intégrité du fichier et assurez-vous et corriger l'anomalie.


In [15]:
fp_map_prod = Path(workspace.files_path) / Path(fp_map_prod)

# (
#    Path(workspace.files_path)
#    / "Fichier Suivi de Stock/data/Mapping produits QAT SAGE X3"
#    / Path(fp_map_prod).name
# )

In [ ]:
assert fp_map_prod.exists(), (
    "Le fichier de mapping des produits n'a pas été retrouvé dans le dossier dédié"
)

## 📌9. Importation du `Plan d'approvisionnement`

- **Format requis :** Respectez la structure du fichier définie pour l’importation peut conserver le fichier brut extrait de QAT.
- **Emplacement du fichier :** Le fichier doit être placé dans le répertoire suivant :  
  **`Fichier Suivi de Stock/data/<programme>/Plan d'Approvisionnement`**
- **En cas d'erreur :**
  - Vérifiez que le fichier est bien présent dans le répertoire.
  - Assurez-vous que les données respectent le format attendu.
  - Vérifiez l'intégrité du fichier et assurez-vous et corriger l'anomalie.


In [17]:
fp_plan_approv = (
    Path(workspace.files_path)
    / f"Fichier Suivi de Stock/data/{programme}/Plan d'Approvisionnement"
    / Path(fp_plan_approv)
)

In [ ]:
df_plan_approv = compute_indicators.file_utils.process_pa_files(
    fp_plan_approv, fp_map_prod, programme, date_report
)

df_plan_approv["facteur_de_conversion_qat_sage"] = df_plan_approv[
    "facteur_de_conversion_qat_sage"
].apply(lambda x: x if not pd.isna(x) else 1)

df_plan_approv.head(2)

,ID de produit QAT,Produits,ID de l`envoi QAT,Commande d`urgence,Commande PGI,Approvisionnement local,N° de commande de l`agent d`approvisionnement,Agent d`approvisionnement,Source de financement,Budget,Status,Quantite,DATE,Cout des Produits,Couts du fret,Couts totaux,Notes,version_pa,date_extraction_pa,Standard product code,facteur_de_conversion_qat_sage,acronym,cout_unitaire_moyen_qat,code_and_date_concate
0,1366,Artenimol/Piperaquine 40/320 mg Tablet 9 Blis...,160667,false,false,false,570-PA-2023,Govt,Govt,8355,Reçu,48063.0,2025-02-05,95164.74,11419.77,106584.51,Livré à la Nouvelle PSP le 15 janvier 2025,44,2025-09-05,3050200.0,1.0,Piperaquine/dihydroartemisinine 320/40 mg 3 tab,1.98,3050200_2025-02-05
1,4660,Oral Rehydration Salts 20.5 gm/L + Zinc Sulfat...,160689,false,false,false,624,Govt,Govt,8355,Expédié,25000.0,2025-08-29,7000.00,840.00,7840.00,"25000 issu du plan d'approvisionnement 2022, a...",44,2025-09-05,3230032.0,1.0,KIT DE SELS DE REHYDRATATION ORAL BP 20.5g sac...,0.28,3230032_2025-08-29


# IV.📊Calcul des Indicateurs


## 📌1. `Etat de Stock (1/2)`


In [19]:
stock_sync_manager.initialize_database_connection()

schema_name = "suivi_stock"

df_etat_stock = stock_sync_manager.get_table_data(
    query=QUERY_ETAT_STOCK.format(
        schema_name=schema_name, programme=programme, date_report_prec=date_report_prec
    )
).drop_duplicates()

df_etat_stock.head(2)

Connexion à la base de données établie avec succès


,id_dim_produit_stock_track_pk,code_produit,ancien_code,categorie,designation,type_produit,unit_niveau_central,unit_niveau_peripherique,facteur_de_conversion,designation_acronym,code_qat,programme,cout_unitaire_moyen_qat,facteur_de_conversion_qat_sage,stock_theorique_mois_precedent
0,1,3050015,AY02027,Médicaments,"AMODIAQUINE/ARTESUNATE 25 / 67,5 mg ENFANT (0 ...",Non traceur,BOITE/25,PLAQUETTE,25,AA (0-11 MOIS),1381.0,PNLP,4.007185,1.0,-112.0
1,2,3050016,AY02028,Médicaments,AMODIAQUINE/ARTESUNATE 50 / 135 mg ENFANT (1 -...,Non traceur,BOITE/25,PLAQUETTE,25,AA (1-5 ANS),1382.0,PNLP,6.237433,1.0,26386.0


In [ ]:
%%time
df_etat_stock = compute_indicators.annexe_1.get_etat_stock_current_month(
    df_etat_stock.copy(),
    df_stock_detaille.copy(),
    df_distribution.copy(),
    df_ppi.copy(),
    df_prelevement.copy(),
    df_receptions.copy(),
    date_report,
)

,id_dim_produit_stock_track_pk,code_produit,ancien_code,categorie,designation,type_produit,unit_niveau_central,unit_niveau_peripherique,facteur_de_conversion,designation_acronym,code_qat,programme,cout_unitaire_moyen_qat,facteur_de_conversion_qat_sage,stock_theorique_mois_precedent,Distribution effectuée,Quantité reçue entrée en stock,Quantité de PPI,Quantité prélévée en Contrôle Qualité (CQ),Ajustement de stock,Stock Théorique Final SAGE,Stock Théorique Final Attendu,ECARTS,Justification des écarts,Diligences,Dilig. Choisie
0,1,3050015,AY02027,Médicaments,"AMODIAQUINE/ARTESUNATE 25 / 67,5 mg ENFANT (0 ...",Non traceur,BOITE/25,PLAQUETTE,25,AA (0-11 MOIS),1381.0,PNLP,4.007185,1.0,-112.0,0,0,0,0,NaN,0,-112.0,112.0,,,
1,2,3050016,AY02028,Médicaments,AMODIAQUINE/ARTESUNATE 50 / 135 mg ENFANT (1 -...,Non traceur,BOITE/25,PLAQUETTE,25,AA (1-5 ANS),1382.0,PNLP,6.237433,1.0,26386.0,407,5702,0,0,NaN,32736,31681.0,1055.0,,,
2,3,3050456,None,Médicaments,ARTEMETHER/ LUMEFANTRINE 80/480mg PLAQ/6 comp....,Non traceur,BOITE/30,None,30,None,None,PNLP,NaN,NaN,3985.0,540,0,0,0,NaN,3429,3445.0,-16.0,,,


CPU times: user 152 ms, sys: 20.9 ms, total: 173 ms
Wall time: 163 ms


## 📌2.`Distributions et Consommations` - `Annexe 1`


### 2.1.`Distributions`


In [21]:
%%time
df_dmm_curent, df_dmm_histo = compute_indicators.annexe_1.get_dmm_current_month(
    df_etat_stock.copy(),
    programme,
    date_report,
    stock_sync_manager.civ_engine,
    schema_name,
    auto_computed_dmm,
)

CPU times: user 99.6 ms, sys: 4.45 ms, total: 104 ms
Wall time: 155 ms


### 2.2.`Consommations`


In [ ]:
# Pour avoir la CMM du mois courant une recherche est d'abord faite sur la feuille StockParRegion provenant du RapportFeedback
eomonth = (pd.to_datetime(date_report) + pd.offsets.MonthEnd(0)).strftime("%Y-%m-%d")

df_stock_prog_nat = stock_sync_manager.get_table_data(
    query=QUERY_ETAT_STOCK_PROGRAMME.format(eomonth=eomonth, programme=programme)
)

df_stock_prog_nat["Code_produit"] = df_stock_prog_nat["Code_produit"].astype(int)

df_stock_prog_nat.head(3)

,Programme,MSD,STATUT,CONSO,SDU,CMM,dispo_globale,dispo_globale_cible,dispo_traceur,dispo_traceur_cible,date_report,id_produit_fk,Code_region,region_order,statut_pourcentage,id_produit_pk,Code_produit,Produit_designation,Unit_rapportage,Categorie_produit,Categorie_du_produit,Code_sous_prog
0,PNLP,"0,6",SOUS-STOCK,89717,107533,167557.50,0.011396,0.02125,0.014679,0.02375,2025-08-31,613,NAT,None,0.025,613,3050016,ARTESUNATE/AMODIAQUINE 50 / 135 mg ENFANT (1 -...,PLAQUETTE,Produit non traceur,PRODUITS PNLP,PNLP-1
1,PNLP,"0,1",SOUS-STOCK,6673,5019,81888.25,0.011396,0.02125,0.014679,0.02375,2025-08-31,17,NAT,None,0.025,17,3050061,ARTEMETHER/LUMEFANTRINE 20 / 120 mg (0 - 3ANS)...,PLAQUETTE,Produit non traceur,PRODUITS PNLP,PNLP-1
2,PNLP,"0,5",SOUS-STOCK,80541,77522,143358.75,0.011396,0.02125,0.014679,0.02375,2025-08-31,4796,NAT,None,0.025,4796,3050062,ARTEMETHER/LUMEFANTRINE 20 / 120 mg (3 - 8 ANS...,PLAQUETTE,Produit non traceur,PRODUITS PNLP,PNLP-1


In [23]:
%%time
df_cmm_curent, df_cmm_histo = compute_indicators.annexe_1.get_cmm_current_month(
    df_etat_stock.copy(),
    df_stock_prog_nat.copy(),
    programme,
    date_report,
    stock_sync_manager.civ_engine,
    schema_name,
    auto_computed_cmm,
)

CPU times: user 136 ms, sys: 4.96 ms, total: 141 ms
Wall time: 194 ms


## 📌3. Calcul des indicateurs `Etat de stock (2/2)` `Annexe 2 - Suivi des Stocks`


In [24]:
df_etat_stock_periph = stock_sync_manager.get_table_data(
    query=QUERY_ETAT_STOCK_PERIPH.format(eomonth=eomonth)
)

df_etat_stock_periph = df_etat_stock_periph.loc[
    df_etat_stock_periph.Code_sous_prog.str.contains(programme)
]

df_etat_stock_periph["Code_produit"] = df_etat_stock_periph["Code_produit"].astype(int)

df_etat_stock_periph.head(2)

,Periode,Code_ets,stock_initial,qte_recue,qte_utilisee,perte_ajust,j_rupture,sdu,cmm_esigl,cmm_gest,qte_prop,qte_cmde,qte_approuv,msd,etat_stock,besoin_cmde_urg,besoin_trsf_in,qte_trsf_out,date_report,qte_cmde_mois_prec,etat_stock_mois_prec,id_produit_fk,id_produit_pk,Code_produit,Produit_designation,Unit_rapportage,Categorie_produit,Categorie_du_produit,Code_sous_prog
69,AOUT 2025,40300050,0,0,0,0,31,0,0,0.0,0,0.0,0,NaN,RUPTURE,0.0,0.0,NaN,2025-08-31,NaN,None,32,32,3050198,DIHYDROARTEMISININE / PIPERAQUINE PHOSPHATE 20...,PLAQUETTE,Produit non traceur,PRODUITS PNLP,PNLP-1
70,AOUT 2025,40300050,0,0,0,0,31,0,0,0.0,0,0.0,0,NaN,RUPTURE,0.0,0.0,NaN,2025-08-31,NaN,None,33,33,3050199,DIHYDROARTEMISININE / PIPERAQUINE PHOSPHATE 40...,PLAQUETTE,Produit non traceur,PRODUITS PNLP,PNLP-1


In [ ]:
df_plan_approv["Quantité harmonisée (SAGE)"] = (
    df_plan_approv["Quantite"] * df_plan_approv["facteur_de_conversion_qat_sage"]
)

In [26]:
%%time
df_etat_stock = compute_indicators.annexe_2(
    df_etat_stock,
    df_dmm_curent.copy(),
    df_stock_prog_nat.copy(),
    df_etat_stock_periph.copy(),
    df_stock_detaille.copy(),
    df_receptions.copy(),
    df_plan_approv.copy(),
    date_report,
)

,id_dim_produit_stock_track_fk,stock_theorique_mois_precedent,distribution_effectuee,quantite_recue_stock,quantite_ppi,quantite_prelevee_cq,ajustement_stock,stock_theorique_final_sage,stock_theorique_final_attendu,ecarts,justification_ecarts,diligences,sdu_central_annexe_2,dmm_central_annexe_2,msd_central_annexe_2,statut_central_annexe_2,conso_decentralise_annexe_2,sdu_decentralise_annexe_2,cmm_decentralise_annexe_2,msd_decentralise_annexe_2,statut_decentralise_annexe_2,nombre_de_site_en_rupture_annexe_2,sdu_national_annexe_2,cmm_national_annexe_2,msd_national_annexe_2,statut_national_annexe_2,date_peremption_plus_proche_brute_annexe_2,date_peremption_plus_proche_annexe_2,quantite_correspondante_annexe_2,msd_correspondant_annexe_2,quantite_attendue_annexe_2,msd_attendu_annexe_2,quantite_non_stockee_annexe_2,msd_recu_annexe_2,financement_annexe_2,date_probable_livraison_annexe_2,date_effective_livraison_annexe_2,statut_annexe_2,analyse_risque_commentaires_annexe_2,diligences_central_annexe_2,diligences_peripherique_annexe_2,responsable_annexe_2,dilig_choisie_annexe_2,date_report
0,1,-112.0,0,0,0,0,NaN,0,-112.0,112.0,,,0,2796.0,"0,0",Rupture,707,556,2027,"0,3",Sous-Stock,129,556,2027,"0,3",Sous-Stock,NaT,NaT,NaN,,15100.0,"5,4",0,"0,0",,2025-09-30,NaT,Expédié,,,,,,2025-08-01
1,2,26386.0,407,5702,0,0,NaN,32736,31681.0,1055.0,,,32736,2490.4,"13,1",SurStock,3589,4302,6703,"0,6",Sous-Stock,92,37038,6703,"5,5",Bien Stocké,2026-06-01,2026-06-01,441.0,"0,2",32000.0,"12,8",5702,"2,3",,2026-01-30,2025-08-13,Approuvé,,,,,,2025-08-01
2,3,3985.0,540,0,0,0,NaN,3429,3445.0,-16.0,,,3429,52.5,"65,3",SurStock,330,283,7000,"0,0",Sous-Stock,205,3712,7000,"0,5",Sous-Stock,2027-05-01,2027-05-01,3429.0,"65,3",0.0,"0,0",0,"0,0",,NaT,NaT,,,,,,,2025-08-01


CPU times: user 459 ms, sys: 820 μs, total: 460 ms
Wall time: 460 ms


# V.Exportation des données

Avant d’exporter les données, il est essentiel de s’assurer qu’aucune information relative à la période en cours n’est déjà présente dans les différentes tables.  
Si des données existent, une suppression devra être effectuée avant l’export.


## 1. `Etat de Stock`


In [27]:
id_list = list(map(int, df_etat_stock.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track
WHERE date_report = '{date_report}' AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, list(id_list))
stock_sync_manager.conn.commit()

In [28]:
assert df_etat_stock.date_report.unique().shape[0] == 1

stock_sync_manager.insert_dataframe_to_table(df_etat_stock, "stock_track")

'Insertion de 39 enrégistrements réussie'

## 2. `Distributions`


### 2.1.`Distributions mois courant`


In [29]:
id_list = list(map(int, df_dmm_curent.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_dmm
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [30]:
assert df_dmm_curent.date_report.unique().shape[0] == 1

stock_sync_manager.insert_dataframe_to_table(df_dmm_curent, "stock_track_dmm")

'Insertion de 39 enrégistrements réussie'

### 2.2.`Distributions hitoriques considérées pour le mois en cours`


In [31]:
id_list = list(map(int, df_dmm_histo.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_dmm_histo
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [32]:
assert df_dmm_histo.date_report.unique().shape[0] == 1

stock_sync_manager.insert_dataframe_to_table(df_dmm_histo, "stock_track_dmm_histo")

'Insertion de 208 enrégistrements réussie'

## 3.`Consommations`


### 3.1. `Consommations mois courant`


In [33]:
id_list = list(map(int, df_cmm_curent.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_cmm
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [34]:
assert df_cmm_curent.date_report.unique().shape[0] == 1

stock_sync_manager.insert_dataframe_to_table(df_cmm_curent, "stock_track_cmm")

'Insertion de 39 enrégistrements réussie'

### 3.2.`Consommations hitoriques considérées pour le mois en cours`


In [35]:
id_list = list(map(int, df_cmm_histo.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_cmm_histo
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [36]:
assert df_dmm_histo.date_report.unique().shape[0] == 1

stock_sync_manager.insert_dataframe_to_table(df_cmm_histo, "stock_track_cmm_histo")

'Insertion de 195 enrégistrements réussie'

## 4.`Stock détaillé`


In [37]:
dim_produit = pd.read_sql(
    f"SELECT * FROM {schema_name}.dim_produit_stock_track where programme='{programme}'",
    stock_sync_manager.civ_engine,
)

In [38]:
code_col = [col for col in df_stock_detaille.columns if "CODE" in str(col).upper()][0]

df_stock_detaille.rename(
    columns={
        code_col: "code_produit",
        "Désignation": "designation_produit",
        "Emplacement": "emplacement",
        "Date limite de consommation": "date_limite_consommation",
        "Numéro Lot": "numero_lot",
        "Sous lot (Programme)": "sous_lot_programme",
        "Qté \nPhysique": "qte_physique",
        "Qté \nlivrable": "qte_livrable",
        "Unit": "unit",
    },
    inplace=True,
)
df_stock_detaille = df_stock_detaille[
    [
        "code_produit",
        "designation_produit",
        "emplacement",
        "date_limite_consommation",
        "numero_lot",
        "sous_lot_programme",
        "qte_physique",
        "qte_livrable",
        "unit",
    ]
]

df_stock_detaille = df_stock_detaille.loc[
    df_stock_detaille.code_produit.isin(dim_produit.code_produit)
]
df_stock_detaille.head(1)

,code_produit,designation_produit,emplacement,date_limite_consommation,numero_lot,sous_lot_programme,qte_physique,qte_livrable,unit
1,4150554,MILDA IG2 BALLE/50,27Q00A00,2028-01-01,I25225G21,PNLP,2677,2677,BALLE


In [39]:
df_stock_detaille = (
    df_stock_detaille.groupby(["code_produit", "date_limite_consommation"])[
        ["qte_physique", "qte_livrable"]
    ]
    .sum(min_count=1)
    .reset_index()
)

df_stock_detaille = (
    dim_produit[["id_dim_produit_stock_track_pk", "code_produit"]]
    .merge(df_stock_detaille, on="code_produit")
    .rename(columns={"id_dim_produit_stock_track_pk": "id_dim_produit_stock_track_fk"})
    .drop(columns=["code_produit"])
)

df_stock_detaille["date_report"] = pd.to_datetime(date_report)

In [40]:
id_list = list(map(int, df_stock_detaille.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_detaille
WHERE date_report = '{date_report}' AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, list(id_list))
stock_sync_manager.conn.commit()

In [41]:
stock_sync_manager.insert_dataframe_to_table(df_stock_detaille, "stock_track_detaille")

del df_stock_detaille

## 5. `Etat de stock de la NPSP`


In [42]:
query = f"""
DELETE FROM {schema_name}.stock_track_npsp
WHERE date_report = '{date_report}' and programme='{programme}';
"""

stock_sync_manager.civ_cursor.execute(query)
stock_sync_manager.conn.commit()

In [43]:
stock_sync_manager.insert_dataframe_to_table(
    df_etat_stock_npsp, table_name="stock_track_npsp", schema_name="suivi_stock"
)

'Insertion de 51 enrégistrements réussie'

## 6.`Prévision`

<div class="alert alert-block alert-warning">
Avant de procéder au calcul des éléments de prévision, il est essentiel de s’assurer que les informations produits sont à jour.
Cette actualisation doit être effectuée à l’aide du fichier de mapping fourni en paramètre du pipeline.
Ce fichier joue un rôle clé : il établit la correspondance entre les différentes sources de données, garantissant ainsi la cohérence et la fiabilité des informations utilisées pour les prévisions.

</div>


In [44]:
stock_sync_manager.synchronize_product_metadata(df_plan_approv.copy(), programme)

'Aucune mise à jour des données à effectuer sur la table dim_produit'

In [ ]:
df_plan_approv["Date updated"] = df_plan_approv["DATE"].apply(lambda x: x.strftime("%b-%Y"))

In [46]:
df_prevision = compute_indicators.prevision.get_prevision_current_month(
    df_plan_approv.copy(),
    date_report,
    programme,
    stock_sync_manager.civ_engine,
    schema_name="suivi_stock",
)

,id_dim_produit_stock_track_fk,code_produit,ancien_code,stock_central,dmm_central,stock_national,cmm_national,stock_prev_central,stock_prev_national,period_prev,date_report
0,14,3010049,AY13071,70370.0,5384.89,82452.0,4655.0,13,18,2025-08-01,2025-08-01
1,14,3010049,AY13071,NaN,NaN,NaN,NaN,12,17,2025-09-01,2025-08-01
2,14,3010049,AY13071,NaN,NaN,NaN,NaN,11,16,2025-10-01,2025-08-01


In [47]:
id_list = list(map(int, df_prevision.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_prevision
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""
stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [48]:
stock_sync_manager.insert_dataframe_to_table(
    df_prevision.drop(columns=["code_produit", "ancien_code"]), "stock_track_prevision"
)

'Insertion de 507 enrégistrements réussie'

## 7.`Plan d'approvisionnement`


In [49]:
query = f"""
DELETE FROM {schema_name}.plan_approv
WHERE date_report = '{date_report}' and programme='{programme}'
"""
stock_sync_manager.civ_cursor.execute(query)
stock_sync_manager.conn.commit()

In [ ]:
df_pa = df_plan_approv.rename(
    columns={
        "Standard product code": "standard_product_code",
        "ID de produit QAT": "id_produit_qat",
        "Produits": "designation",
        "ID de l`envoi QAT": "id_envoi_qat",
        "Agent d`approvisionnement": "centrale_achat",
        "Source de financement": "source_financement",
        "Status": "status",
        "Quantite": "quantite",
        "facteur_de_conversion_qat_sage": "facteur_conversion_qat_vers_sage",
        "Quantité harmonisée (SAGE)": "quantite_harmonisee_sage",
        "DATE": "date",
        "Cout des Produits": "cout_produits",
        "Couts du fret": "cout_fret",
        "Couts totaux": "cout_total",
    },
)

df_pa = df_pa[
    [
        "standard_product_code",
        "id_produit_qat",
        "designation",
        "id_envoi_qat",
        "centrale_achat",
        "source_financement",
        "status",
        "quantite",
        "facteur_conversion_qat_vers_sage",
        "quantite_harmonisee_sage",
        "date",
        "cout_produits",
        "cout_fret",
        "cout_total",
        "cout_unitaire_moyen_qat",
        "version_pa",
        "date_extraction_pa",
    ]
]
df_pa["date_report"] = pd.to_datetime(date_report)
df_pa["programme"] = programme
df_pa["standard_product_code"] = df_pa["standard_product_code"].fillna(0)

df_pa.head(2)

,standard_product_code,id_produit_qat,designation,id_envoi_qat,centrale_achat,source_financement,status,quantite,facteur_conversion_qat_vers_sage,quantite_harmonisee_sage,date,cout_produits,cout_fret,cout_total,cout_unitaire_moyen_qat,version_pa,date_extraction_pa,date_report,programme
0,3050200.0,1366,Artenimol/Piperaquine 40/320 mg Tablet 9 Blis...,160667,Govt,Govt,Reçu,48063.0,1.0,48063.0,2025-02-05,95164.74,11419.77,106584.51,1.98,44,2025-09-05,2025-08-01,PNLP
1,3230032.0,4660,Oral Rehydration Salts 20.5 gm/L + Zinc Sulfat...,160689,Govt,Govt,Expédié,25000.0,1.0,25000.0,2025-08-29,7000.00,840.00,7840.00,0.28,44,2025-09-05,2025-08-01,PNLP


In [ ]:
stock_sync_manager.insert_dataframe_to_table(
    df_pa, table_name="plan_approv", schema_name="suivi_stock"
)

'Insertion de 99 enrégistrements réussie'

In [52]:
del df_pa

## 8.`Mise à jour des informations sur les produits à partir du plan d'appro`

Cette opération a pour principal but de mettre à jour les informations sur les désignation accronyme issue du fichier de mapping `QAT` et `SAGE X3`.


In [ ]:
df_pa = df_plan_approv[
    [
        "Standard product code",
        "acronym",
        "facteur_de_conversion_qat_sage",
    ]
].drop_duplicates()

stock_sync_manager.synchronize_product_metadata(df_pa, programme)

'Aucune mise à jour des données à effectuer sur la table dim_produit'

In [54]:
del df_pa

## 9.`Mise à jour globale des informations abérantes du type nan, NaN, None`


In [ ]:
# Mise à jour des colonnes qui contienent des valeurs nan, NaN ou None malgré les mises à jours effectuée
from database_operations import queries

stock_sync_manager.civ_cursor.execute(queries.QUERY_UPDATE.format(schema_name=schema_name))
stock_sync_manager.conn.commit()

# VI.📄Génération du fichier pour le mois spécifique


## 1. Initialisation des paramètres utile


In [ ]:
date_report_format = date_report
date_report = pd.to_datetime(date_report, format="%Y-%m-%d").strftime("%d/%m/%Y")
f_month = gstf.get_current_variable(date_report)[1].replace(" ", "-")

## 2. Importation des classeurs de bases


In [59]:
%%time
# WorkBook Template de base
wb_template = pyxl.load_workbook(
    filename="/home/jovyan/workspace/Fichier Suivi de Stock/code/pipelines/generate_stock_tracking_file/Template Fichier Suivi de Stock/Fichier Suivi de Stock Template.xlsx"
)

# WorkBook Etats de stock mensuels
wb_etat_stock = pyxl.load_workbook(filename=fp_etat_mensuel)

# WorkBook du Rapport Feedback
wb_fbr = pyxl.load_workbook(
    filename=f"/home/jovyan/workspace/Rapport Feedback/code/pipelines/rapport feedback genere/{date_report_format[:4]}/Rapport FeedBack-{f_month}.xlsx"
)

CPU times: user 6min 36s, sys: 9.29 s, total: 6min 45s
Wall time: 6min 52s


## 3. Mise à jour des données en se basant sur la source `Etat de Stock Mensuel`


In [ ]:
%%time
wb_template = gstf.update_sheets_etat_mensuel(wb_etat_stock, wb_template, programme, date_report)

CPU times: user 48.4 s, sys: 321 ms, total: 48.8 s
Wall time: 49.5 s


## 4. Mise à jour des données en se basant sur la source `Rapport Feedback`

Pour le faire on va se baser sur les inputs au préalable du fichier RapportFeebback généré et présent dans les fichiers d'OH pour le mois courant


In [62]:
df_etat_stock = pd.read_excel(
    f"/home/jovyan/workspace/Rapport Feedback/code/pipelines/rapport feedback genere/{date_report_format[:4]}/Rapport FeedBack-{f_month}.xlsx",
    sheet_name="ETAT DU STOCK",
)

df_etat_stock.head(2)

,Code_Pro,CODE,PROGRAMME,SOUS-PROGRAMME,PERIODE,REGION,DISTRICT,CODE ETS,STRUCTURE,TYPE DE STRUCTURE,CATEGORIE PRODUIT,PRODUIT,UNITE DE RAPPORTAGE,STOCK INITIAL,QUANTITE RECUE,QUANTITE UTILISEE,PERTES ET AJUSTEMENT,JOURS DE RUPTURE,SDU,CMM ESIGL,CMM gestionnaire,QUANTITE PROPOSEE,QUANTITE COMMANDEE,QUANTITE APPROUVEE,MSD,ETAT DU STOCK,BESOIN COMMANDE URGENTE,BESOIN TRANSFERT IN,QUANTITE A TRANSFERER OUT,CATEGORIE_DU_PRODUIT,Commandes du mois précédent,Etat du stock du mois précédent
0,3180080_PNLS,3180080,PNLS,PNLS-ANTIRETROVIRAUX ET IO,AOUT 2025,ABIDJAN 2,TREICHVILLE-MARCORY,50300070,DISTRICT SANITAIRE TREICHVILLE MARCORY,DISTRICT SANITAIRE,Produit non traceur,VITAMIN B6 ( PYRIDOXINE CHLORHYDRATE)50mg tabl...,COMPRIME,4600,24000,2700,0,0,25900,5000,7725.0,0,5000,0,3.352751,BIEN STOCKE,NaN,NaN,NaN,ARV-ADULTE,NaN,NaN
1,3050257_PNLS,3050257,PNLS,PNLS-ANTIRETROVIRAUX ET IO,AOUT 2025,ABIDJAN 2,TREICHVILLE-MARCORY,50300070,DISTRICT SANITAIRE TREICHVILLE MARCORY,DISTRICT SANITAIRE,Produit non traceur,H 100 mg (ISONIAZIDE) comp. Dispersible BTE/10...,COMPRIME,2500,3000,1900,0,0,3600,1433,1433.0,2132,2132,2132,2.512212,BIEN STOCKE,NaN,NaN,NaN,ARV-ADULTE,NaN,NaN


In [ ]:
%%time

wb_template = gstf.update_sheet_etat_stock(
    wb_template, df_etat_stock.loc[df_etat_stock.PROGRAMME == programme].copy()
)

wb_template = gstf.update_sheet_stock_region(wb_template, wb_fbr, programme)

CPU times: user 13.3 s, sys: 58.8 ms, total: 13.3 s
Wall time: 13.5 s


## 4. Mise à jour des informations feuille `Annexe 1 - Consolidation`


In [65]:
wb_template = gstf.update_sheet_annexe_1(
    wb_template,
    programme,
    schema_name,
    stock_sync_manager.civ_engine,
    date_report,
    auto_computed_dmm,
    auto_computed_cmm,
)

## 5. Mise à jour des données du `Plan d'approvisionnement`


In [ ]:
df_pa = df_plan_approv.rename(
    columns={
        "Agent d`approvisionnement": "Centrale d'achat",
        "Source de financement": "Source Financement",
        "facteur_de_conversion_qat_sage": "Facteur de conversion de QAT vers SAGE",
        "cout_unitaire_moyen_qat": "Coût unitaire moyen (en dollar)",
        "acronym": "Acronym",
    }
)

df_pa["DATE"] = pd.to_datetime(df_pa["DATE"], format="%d-%b-%Y", errors="coerce")

In [67]:
%%time
wb_template = gstf.update_sheet_plan_approv(wb_template, df_pa)

CPU times: user 185 ms, sys: 981 μs, total: 186 ms
Wall time: 186 ms


## 5. Mise à jour des informations en se basant sur la source `Annexe 2 - Suivi de Stock`


In [68]:
%%time
wb_template = gstf.update_sheet_annexe_2(wb_template, df_pa, date_report)

CPU times: user 396 ms, sys: 1.04 ms, total: 397 ms
Wall time: 396 ms


## 6. Mise à jour des données en se basant sur la source `Prévision`


In [ ]:
df_prod = dim_produit[["code_produit", "type_produit", "designation", "designation_acronym"]]

df_prod["designation"] = df_prod.apply(
    lambda row: (
        row["designation_acronym"]
        if not pd.isna(row["designation_acronym"])
        else row["designation"]
    ),
    axis=1,
)
df_prod = df_prod.drop(columns="designation_acronym")

In [70]:
wb_template = gstf.update_sheet_prevision(wb_template, date_report, df_prod)

## 7. Mise à jour de certains éléments de la feuille `Rapport`


In [ ]:
ws_rapport = wb_template["Rapport"]
ws_rapport["D4"].value = programme
ws_rapport["D6"].value = f_month.split("-")[0].capitalize()
ws_rapport["E6"].value = date_report_format[:4]

# VII. Exportation du Fichier Suivi de Stock vers le `drive` et la `BD`


In [72]:
%%time
# Sauvegarde du fichier dans un repertoire courant
dest_file = (
    Path(workspace.files_path)
    / f"Fichier Suivi de Stock/code/pipelines/fichier suivi de stock genere/{programme}/{date_report_format[:4]}"
)

dest_file.mkdir(exist_ok=True, parents=True)

dest_file = dest_file / f"Fichier Suivi de Stock {programme}-{f_month}.xlsx"

wb_template.save(dest_file)

CPU times: user 3.32 s, sys: 108 ms, total: 3.43 s
Wall time: 3.91 s


In [73]:
download_url = ggdrive.upload_and_return_link(
    dest_file.as_posix(),
    date_report_format,
)

Aucun file trouvé avec ce nom.


In [74]:
df_download_url = pd.DataFrame(
    data=[
        {
            "programme": programme,
            "download_url": download_url,
            "date_report": date_report_format,
        }
    ]
)

df_download_url.date_report = df_download_url.date_report.apply(
    lambda x: pd.to_datetime(str(x)[:10], format="%Y-%m-%d")
)

In [75]:
stock_sync_manager.synchronize_table_data(
    df_download_url,
    table_name="share_link",
    merge_keys=["programme", "date_report"],
    programme=programme,
)

Insertion de 1 enrégistrements réussie


'Aucune mise à jour des données à effectuer'